# Importing the Libraries

In [1]:
from keras import backend as K
import time
from multiprocessing.dummy import Pool
K.set_image_data_format('channels_first')
import cv2
import os
import glob
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
import win32com.client as wincl

Using TensorFlow backend.


In [2]:
padding = 50
ready_to_detect_identity = True
windows10_voice_interface = wincl.Dispatch("SAPI.SpVoice")

FRModel=faceRecoModel(input_shape=(3,96,96))

# Function to get Triplet Loss

In [3]:
def triplet_loss(y_true,y_pred,alpha=0.3):
    anchor,positive,negative=y_pred[0],y_pred[1],y_pred[2]
    #Compute the distance between the anchor and positive image.
    pos_dist= tf.reduce_sum(tf.square(tf.subtract(anchor,positive)),axis=-1)
    #Compute the distance between the anchor and the negative image.
    neg_dist=tf.reduce_sum(tf.square(tf.subtract(anchor,negative)),axis=-1)
    #Compute the total loss
    basic_loss=tf.add(tf.subtract(pos_dist,neg_dist),alpha)
    # Compute the maxima
    loss = tf.reduce_sum(tf.maximum(basic_loss,0.0))
    return loss

In [4]:
FRModel.compile(optimizer='adam',loss= triplet_loss,metrics=['accuracy'])
load_weights_from_FaceNet(FRModel)

# Function to Prepare Database

In [5]:
def prepare_database():
    database={}
    for file in glob.glob(r"C:\Users\Rupankar\Desktop\Kaggle\Face_Recognition\Training-data/*"):
        identity=os.path.splitext(os.path.basename(file))[0]
        database[identity]= img_path_to_encoding(file,FRModel)
    return database       

# Function for Face Recognition

In [6]:
def webcam_face_recognizer(database):
    global ready_to_detect_identity
    cv2.namedWindow("Preview")
    vc = cv2.VideoCapture(0)
    face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    while vc.isOpened:
        _,frame=vc.read()
        img=frame
        if ready_to_detect_identity:
            img= process_frame(img,frame,face_cascade)
        key=cv2.waitKey(100)
        cv2.imshow("preview",img)
        if key == 27:
            break
    cv2.destroyWindow("Preview")

# Function for Processing the Frame

In [7]:
def process_frame(img,frame,face_cascade):
    global ready_to_detect_identity
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces= face_cascade.detectMultiScale(gray, 1.3, 5)
    identities=[]
    for (x,y,w,h) in faces:
        x1=x-padding
        y1=y-padding
        x2=x+w+padding
        y2=y+h+padding
        img=cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),2)
        identity=find_identity(frame,x1,y1,x2,y2)
        if identity is not None:
            identities.append(identity)
    if identities != []:
        cv2.imwrite('example.png',img)

        ready_to_detect_identity = False
        pool = Pool(processes=1) 
        # We run this as a separate process so that the camera feedback does not freeze
        pool.apply_async(welcome_users, [identities])
    return img

# Function to Find Identity

In [8]:
def find_identity(frame,x1,y1,x2,y2):
    height,width,channels=frame.shape
    part_image=frame[max(0,y1):min(height,y2),max(0,x1):min(width,x2)]
    return who_is_it(part_image,database,FRModel)

# Function for who is it!!

In [9]:
def who_is_it(image,database,model):
    encoding = img_to_encoding(image, model)
    
    min_dist = 100
    identity = None
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current "emb" from the database.
        dist = np.linalg.norm(db_enc - encoding)

        print('distance for %s is %s' %(name, dist))

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name
        if dist < min_dist:
            min_dist = dist
            identity = name
           
    
    if min_dist > 0.8:
        return None
    else:
        return str(identity)

# Prediction Function

In [ ]:
def welcome_users(identities):
    """ Outputs a welcome audio message to the users """
    global ready_to_detect_identity
    welcome_message = 'Welcome '

    if len(identities) == 1:
        welcome_message += '%s, have a nice day.' % identities[0]
    else:
        for identity_id in range(len(identities)-1):
            welcome_message += '%s, ' % identities[identity_id]
            welcome_message += 'and %s, ' % identities[-1]
            welcome_message += 'have a nice day!'

    windows10_voice_interface.Speak(welcome_message)

    # Allow the program to start detecting identities again
    ready_to_detect_identity = True

if __name__ == "__main__":
    database = prepare_database()
    webcam_face_recognizer(database)



distance for Bibhas Chandra Gangopadhyay is 0.92419344
distance for Mallika Ganguly is 1.140605
distance for Payel Ganguly is 0.6469113
distance for Proteem Ganguly is 1.0666062
distance for Bibhas Chandra Gangopadhyay is 1.0913054
distance for Mallika Ganguly is 1.241184
distance for Payel Ganguly is 0.67196876
distance for Proteem Ganguly is 1.2019899
distance for Bibhas Chandra Gangopadhyay is 0.92199564
distance for Mallika Ganguly is 1.0466665
distance for Payel Ganguly is 0.64758694
distance for Proteem Ganguly is 1.057567
distance for Bibhas Chandra Gangopadhyay is 1.0541924
distance for Mallika Ganguly is 1.114402
distance for Payel Ganguly is 0.864844
distance for Proteem Ganguly is 1.1234862
distance for Bibhas Chandra Gangopadhyay is 0.9516813
distance for Mallika Ganguly is 1.0869026
distance for Payel Ganguly is 0.82812333
distance for Proteem Ganguly is 1.0851027
distance for Bibhas Chandra Gangopadhyay is 1.0318613
distance for Mallika Ganguly is 1.0561118
distance for 

distance for Bibhas Chandra Gangopadhyay is 1.1950195
distance for Mallika Ganguly is 0.99951816
distance for Payel Ganguly is 1.155705
distance for Proteem Ganguly is 1.0293157
distance for Bibhas Chandra Gangopadhyay is 1.175743
distance for Mallika Ganguly is 1.0597334
distance for Payel Ganguly is 1.0870357
distance for Proteem Ganguly is 1.0577328
distance for Bibhas Chandra Gangopadhyay is 1.1886828
distance for Mallika Ganguly is 1.023262
distance for Payel Ganguly is 1.1094121
distance for Proteem Ganguly is 1.0426602
distance for Bibhas Chandra Gangopadhyay is 1.1767181
distance for Mallika Ganguly is 1.0138713
distance for Payel Ganguly is 1.077016
distance for Proteem Ganguly is 1.0472188
distance for Bibhas Chandra Gangopadhyay is 1.2173624
distance for Mallika Ganguly is 1.0125105
distance for Payel Ganguly is 1.0983994
distance for Proteem Ganguly is 1.0908966
distance for Bibhas Chandra Gangopadhyay is 1.1572422
distance for Mallika Ganguly is 0.9506319
distance for Paye

distance for Bibhas Chandra Gangopadhyay is 1.0404834
distance for Mallika Ganguly is 0.99475145
distance for Payel Ganguly is 0.9199555
distance for Proteem Ganguly is 0.98312366
distance for Bibhas Chandra Gangopadhyay is 1.044638
distance for Mallika Ganguly is 0.9061157
distance for Payel Ganguly is 1.0446397
distance for Proteem Ganguly is 0.93148434
distance for Bibhas Chandra Gangopadhyay is 1.0062445
distance for Mallika Ganguly is 0.9460916
distance for Payel Ganguly is 0.9761847
distance for Proteem Ganguly is 0.91678077
distance for Bibhas Chandra Gangopadhyay is 0.9721774
distance for Mallika Ganguly is 0.9164847
distance for Payel Ganguly is 0.9992598
distance for Proteem Ganguly is 0.88212425
distance for Bibhas Chandra Gangopadhyay is 1.016392
distance for Mallika Ganguly is 0.9121036
distance for Payel Ganguly is 1.0458984
distance for Proteem Ganguly is 0.8885314
distance for Bibhas Chandra Gangopadhyay is 1.0038382
distance for Mallika Ganguly is 0.9005619
distance fo

distance for Bibhas Chandra Gangopadhyay is 0.8886261
distance for Mallika Ganguly is 1.0515213
distance for Payel Ganguly is 0.720461
distance for Proteem Ganguly is 1.0735525
distance for Bibhas Chandra Gangopadhyay is 1.0172331
distance for Mallika Ganguly is 1.1322929
distance for Payel Ganguly is 0.7752689
distance for Proteem Ganguly is 1.1769289
distance for Bibhas Chandra Gangopadhyay is 1.0287203
distance for Mallika Ganguly is 1.1171864
distance for Payel Ganguly is 0.80472904
distance for Proteem Ganguly is 1.180004
distance for Bibhas Chandra Gangopadhyay is 1.0233319
distance for Mallika Ganguly is 1.1552907
distance for Payel Ganguly is 0.79759985
distance for Proteem Ganguly is 1.1956394
distance for Bibhas Chandra Gangopadhyay is 0.96883583
distance for Mallika Ganguly is 1.1207563
distance for Payel Ganguly is 0.7603878
distance for Proteem Ganguly is 1.1308562
distance for Bibhas Chandra Gangopadhyay is 1.060437
distance for Mallika Ganguly is 1.1625222
distance for P